In [1]:
!pip install bs4
!pip install requests

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page

### Load data and Helper methods 

In [3]:
# Download csvs from my drive
!gdown --id 1f-40Rc7fglPiGXgsSWC87pxmteNS19bS --output companies.csv
!gdown --id 1gjRSdB9oYTaaFszI729ECh8fAy7WcU2m --output rounds2.csv
!gdown --id 1OX0D16HMIpOJiSdbCO_G3YC1vCCr8qRK --output mapping.csv

Downloading...
From: https://drive.google.com/uc?id=1f-40Rc7fglPiGXgsSWC87pxmteNS19bS
To: /content/companies.csv
8.96MB [00:00, 41.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gjRSdB9oYTaaFszI729ECh8fAy7WcU2m
To: /content/rounds2.csv
11.8MB [00:00, 25.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1OX0D16HMIpOJiSdbCO_G3YC1vCCr8qRK
To: /content/mapping.csv
100% 23.0k/23.0k [00:00<00:00, 8.50MB/s]


In [4]:
#Create companies data frame
companies = pd.read_csv('companies.csv', encoding = 'ISO-8859-1')
companies.permalink = companies.permalink.str.encode('ISO-8859-1').str.decode('ascii', 'ignore')

In [5]:
#Create rounds data frame
rounds = pd.read_csv('rounds2.csv', encoding = 'ISO-8859-1')
rounds.company_permalink = rounds.company_permalink.str.encode('ISO-8859-1').str.decode('ascii', 'ignore')

In [6]:
#Create sectors data frame
sectors = pd.read_csv('mapping.csv', encoding = 'ISO-8859-1')

In [7]:
#This function will change the float value to its closest Thousand, Million value.
import math

millnames = ['',' Thousand',' Million',' Billion',' Trillion']

def millify(n):
    n = float(n)
    millidx = max(0,min(len(millnames)-1,
                        int(math.floor(0 if n == 0 else math.log10(abs(n))/3))))

    return '{:.0f}{}'.format(n / 10**(3 * millidx), millnames[millidx])

In [8]:
#This method will remove outliers
def get_iqr_range(df, col=0):
  if(col != 0):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    iqr = Q3-Q1 
    return Q1, Q3,iqr 
  else:
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    iqr = Q3-Q1 
    return Q1[0], Q3[0],iqr[0] 

In [9]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)

### Get Country list where English is primary or official language

In [10]:
# Read html tables from the url
# Language wiki page to check is the Primary or Official language is English
dataframe_list = pd.read_html('https://en.wikipedia.org/wiki/List_of_countries_and_territories_where_English_is_an_official_language', flavor='bs4')

# Country code wiki page to check the country_code for countries where it is missing in above page, 
#we will need the country code to verify if the country code present in master_frame have a primary/official lang as English or not.
country_code_list = pd.read_html('https://www.iban.com/country-codes', flavor='bs4')

In [11]:
# These 3 tables have data about primary language
df1 = dataframe_list[0]
df2 = dataframe_list[1]
df3 = dataframe_list[2]

# Concatenate the above 3 tables to get all the data
english_countries = pd.concat([df1,df2,df3])

# Check the unique Country names
english_countries.Country.unique()

array(['Australia', 'New Zealand[1]', 'United Kingdom', 'United States',
       'Antigua and Barbuda[2]', 'Bahamas[2]', 'Barbados[3]', 'Belize[4]',
       'Botswana[4]', 'Burundi[5]', 'Cameroon[2]', 'Canada[2]',
       'Dominica[2]', 'Eswatini[2]', 'Fiji[2]', 'Gambia[2]', 'Ghana[2]',
       'Grenada[2]', 'Guyana[6]', 'India[4][7]', 'Ireland[8][9]',
       'Jamaica[10]', 'Kenya[2]', 'Kiribati[2]', 'Lesotho[2]',
       'Liberia[2]', 'Malawi[11]', 'Malta[2]', 'Marshall Islands[2]',
       'Mauritius[2]', 'Micronesia[2]', 'Namibia[2]', 'Nauru[12]',
       'Nigeria[2][13]', 'Pakistan[2]', 'Palau[4]',
       'Papua New Guinea[14][15]', 'Philippines[2][16]17', 'Rwanda[2]',
       'Saint Kitts and Nevis[17]', 'Saint Lucia[2]',
       'Saint Vincent and the Grenadines[18]', 'Samoa[19]',
       'Seychelles[2]', 'Sierra Leone[2]', 'Singapore[20][21]',
       'Solomon Islands[2]', 'South Africa[23]', 'South Sudan[24]',
       'Sudan[2]', 'Tanzania[2]', 'Tonga[25]', 'Trinidad and Tobago[2]',
      

In [12]:
# Clean up the country names as it has [] in some of the values
english_countries.Country = english_countries.Country.apply(lambda x: x.split('[')[0])

# Check the unique Country names after removing []
english_countries.Country.unique()

array(['Australia', 'New Zealand', 'United Kingdom', 'United States',
       'Antigua and Barbuda', 'Bahamas', 'Barbados', 'Belize', 'Botswana',
       'Burundi', 'Cameroon', 'Canada', 'Dominica', 'Eswatini', 'Fiji',
       'Gambia', 'Ghana', 'Grenada', 'Guyana', 'India', 'Ireland',
       'Jamaica', 'Kenya', 'Kiribati', 'Lesotho', 'Liberia', 'Malawi',
       'Malta', 'Marshall Islands', 'Mauritius', 'Micronesia', 'Namibia',
       'Nauru', 'Nigeria', 'Pakistan', 'Palau', 'Papua New Guinea',
       'Philippines', 'Rwanda', 'Saint Kitts and Nevis', 'Saint Lucia',
       'Saint Vincent and the Grenadines', 'Samoa', 'Seychelles',
       'Sierra Leone', 'Singapore', 'Solomon Islands', 'South Africa',
       'South Sudan', 'Sudan', 'Tanzania', 'Tonga', 'Trinidad and Tobago',
       'Tuvalu', 'Uganda', 'Vanuatu', 'Zambia', 'Zimbabwe', 'Bahrain',
       'Bangladesh', 'Brunei', 'Cambodia', 'Cyprus', 'Eritrea',
       'Ethiopia', 'Israel', 'Jordan', 'Kuwait', 'Malaysia', 'Maldives',
       'Mya

In [13]:
# Drop the unecessary columns and set column names
english_countries.drop(['No.','Region','Population','Population1'], axis=1, inplace=True)
english_countries.columns = ['Country', 'country_code', 'Primary_English']
english_countries.head()

,Country,country_code,Primary_English
0,Australia,AUS,Yes
1,New Zealand,NZL,Yes
2,United Kingdom,GBR,Yes
3,United States,USA,Yes
0,Antigua and Barbuda,ATG,Yes


In [14]:
# As we can see there are countries in the english_countries dataframe which do not have country code present we will get country code from country_codes df
# We need country code as companies.csv have country_code and to check if the country has primary or official language as English 
# we will need to map the master_frame df with english_countries df.
country_codes = country_code_list[0]

In [15]:
# Drop unecessary columns
country_codes.drop(['Alpha-2 code','Numeric'], axis=1, inplace=True)

# Rename columns 
country_codes.columns = ['Country', 'country_code']
country_codes.head()

,Country,country_code
0,Afghanistan,AFG
1,Åland Islands,ALA
2,Albania,ALB
3,Algeria,DZA
4,American Samoa,ASM


In [16]:
# There are some country_code as NAN, but we have data for those country code in master_frame data frame, so will need to get data for these
english_countries = pd.merge(english_countries,
                 country_codes, on='Country',how='left')

In [17]:
# Fill the NANs in country code with the data from country list
english_countries.country_code_x.fillna(english_countries.country_code_y, inplace=True)

# Drop the merged columns with country code.
english_countries.drop(['country_code_y'], axis=1, inplace=True)

#Change column names to make sense
english_countries.columns = ['Country', 'country_code', 'Primary_English']

#Drop NANs
english_countries.dropna(subset=['country_code'], inplace=True)

In [18]:
# Check all the status for is Primary_English, this we will use in next step to filter countries where primary or official language is not english
english_countries.Primary_English.unique()

array(['Yes', 'No', 'Yes (except Quebec, northern New Brunswick)',
       'Yes (used as lingua franca, mostly and widely spoken, educational, commerce, and government)',
       'Yes (used as lingua franca)',
       'Yes (except for small French Creole population)',
       'No (but official and educational)', 'Yes (Irish is co-official)',
       'Yes (in business and education)',
       'No (but official and in business / education)',
       'No (used as lingua franca)', 'No (but widely spoken)',
       'Yes (used as lingua franca, mostly and widely spoken, and educational)',
       'Yes (and official, educational and lingua franca in formal economy)',
       'No (but official and educational)[26]', nan], dtype=object)

In [19]:
# If the Primary_English is NAN that means the official language is not english hence we will set the NANs to No
english_countries.Primary_English.fillna('No', inplace=True)

# Drop No, No (used as lingua franca), No (but widely spoken) as these means that English is not there Primary or Official language
english_countries = english_countries[~(english_countries.Primary_English.isin(['No', 'No (used as lingua franca)', 'No (but widely spoken)']))]
english_countries.head()

,Country,country_code,Primary_English
0,Australia,AUS,Yes
1,New Zealand,NZL,Yes
2,United Kingdom,GBR,Yes
3,United States,USA,Yes
4,Antigua and Barbuda,ATG,Yes


In [20]:
# Verify if any NANs in country_code
english_countries[english_countries['country_code'].isna()].country_code

Series([], Name: country_code, dtype: object)

In [21]:
# We will use this list to filter the data for investment as Sparks Fund only wants to invest only in English-speaking countries because of the ease of communication with the companies it would invest in
english_countries_list = english_countries['country_code'].tolist()

In [22]:
str(english_countries_list)

"['AUS', 'NZL', 'GBR', 'USA', 'ATG', 'BHS', 'BRB', 'BLZ', 'CAN', 'DMA', 'FJI', 'GHA', 'GRD', 'GUY', 'IND', 'IRL', 'JAM', 'KEN', 'LBR', 'MLT', 'NGA', 'PAK', 'PNG', 'PHL', 'RWA', 'KNA', 'LCA', 'VCT', 'SLE', 'SGP', 'ZAF', 'TTO', 'UGA']"

### Companies data

In [23]:
companies.head(10)

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/Organization/-Fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/Organization/-Qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/Organization/-The-One-Of-Them-Inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
3,/Organization/0-6-Com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007
4,/Organization/004-Technologies,004 Technologies,http://004gmbh.de/en/004-interact,Software,operating,USA,IL,"Springfield, Illinois",Champaign,01-01-2010
5,/Organization/01Games-Technology,01Games Technology,http://www.01games.hk/,Games,operating,HKG,NaN,Hong Kong,Hong Kong,NaN
6,/Organization/0Ndine-Biomedical-Inc,Ondine Biomedical Inc.,http://ondinebio.com,Biotechnology,operating,CAN,BC,Vancouver,Vancouver,01-01-1997
7,/Organization/0Xdata,H2O.ai,http://h2o.ai/,Analytics,operating,USA,CA,SF Bay Area,Mountain View,01-01-2011
8,/Organization/1,One Inc.,http://whatis1.com,Mobile,operating,USA,CA,SF Bay Area,San Francisco,01-08-2011
9,/Organization/1-2-3-Listo,"1,2,3 Listo",http://www.123listo.com,E-Commerce,operating,CHL,12,Santiago,Las Condes,01-01-2012


In [24]:
#Check for the columns and row size for the dataframe
companies.shape

(66368, 10)

In [25]:
#Check the row size for the permalink column
companies.permalink.shape

(66368,)

In [26]:
#Check number of null values in permalink col
companies.permalink.isnull().sum()

0

In [27]:
#By looking at data i can see different casing has been used, to avoid mistake just convert the column to upper case
#We will do the same for rounds company_permalink column
companies.permalink = companies.permalink.apply(lambda x: x.upper())

How many unique companies are present in companies?

In [28]:
companies.permalink.nunique()

66368

### Rounds data

In [29]:
rounds.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.000
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,nan
2,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.000
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.000
4,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.000


In [30]:
#By looking at data i can see different casing has been used, to avoid mistake just convert the column to upper case
#We have done the same for companies permalink column
rounds.company_permalink = rounds.company_permalink.apply(lambda x: x.upper())

In [31]:
#Check for number of rows and columns in the dataframe
rounds.shape

(114949, 6)

In [32]:
#Check number of null values in company_permalink col
rounds.company_permalink.isnull().sum()

0

How many unique companies are present in rounds2?

In [33]:
rounds.company_permalink.nunique()

66368

In [34]:
# verifying the list of entities that are there in rounds but not in companies
print(len(rounds.loc[~rounds['company_permalink'].isin(companies['permalink']), :]))

0


### Master dataframe

In [35]:
master_frame = pd.merge(rounds,
                 companies, left_on='company_permalink',right_on='permalink',how='left')

In [36]:
master_frame.head()

,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.000,/ORGANIZATION/-FAME,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN
1,/ORGANIZATION/-QOUNTER,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,nan,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
2,/ORGANIZATION/-QOUNTER,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.000,/ORGANIZATION/-QOUNTER,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014
3,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.000,/ORGANIZATION/-THE-ONE-OF-THEM-INC-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN
4,/ORGANIZATION/0-6-COM,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.000,/ORGANIZATION/0-6-COM,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007


In [37]:
master_frame['permalink'].isnull().sum()

0

In [38]:
# Number of observations in master_frame df
master_frame.company_permalink.isnull().value_counts()

False    114949
Name: company_permalink, dtype: int64

In [39]:
# Drop the columns we don't need
master_frame.drop(['funding_round_code', 'homepage_url','status','state_code','region','city','founded_at'], axis=1, inplace=True)

In [40]:
# Are there any null values
master_frame.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funded_at                      0
raised_amount_usd          19990
permalink                      0
name                           1
category_list               3410
country_code                8678
dtype: int64

In [41]:
# Remove null from category list and country code column
master_frame.dropna(subset=['category_list','country_code','raised_amount_usd','name'], inplace=True)

In [42]:
# After dropping the columns do we still have missing values?
master_frame.isnull().sum()

company_permalink          0
funding_round_permalink    0
funding_round_type         0
funded_at                  0
raised_amount_usd          0
permalink                  0
name                       0
category_list              0
country_code               0
dtype: int64

In [43]:
#As we are only interested in below 4 investment type, get the data only for those
master_frame = master_frame[master_frame.funding_round_type.isin(['venture','seed','angel','private_equity'])]

In [44]:
master_frame.shape

(75123, 9)

In [45]:
# Considering the constraints of Sparks Fund, that Sparks Fund only wants to invest only in English-speaking countries because of the ease of communication with the companies it would invest in
master_frame = master_frame[master_frame['country_code'].isin(english_countries_list)]

In [46]:
master_frame.shape

(61300, 9)

In [47]:
master_frame.country_code.unique()

array(['IND', 'USA', 'CAN', 'GBR', 'AUS', 'IRL', 'SGP', 'NGA', 'NZL',
       'KEN', 'ZAF', 'PHL', 'UGA', 'PAK', 'GHA', 'BRB', 'TTO', 'MLT',
       'JAM', 'KNA', 'RWA', 'DMA', 'GRD'], dtype=object)

In [48]:
# Considering the constraints of Spark Funds, that spark fund only want to invest between 5 to 15 million, filter out the rest data
master_frame = master_frame[((master_frame['raised_amount_usd']>5000000) & (master_frame['raised_amount_usd']<15000000))]

In [49]:
master_frame.shape

(11810, 9)

In [50]:
master_frame.funding_round_type.value_counts()

venture           11353
private_equity      302
seed                119
angel                36
Name: funding_round_type, dtype: int64

In [51]:
master_frame.groupby(by='funding_round_type')['raised_amount_usd'].describe()

,count,mean,std,min,25%,50%,75%,max
funding_round_type,,,,,,,,
angel,36.000,7484446.917,2120123.503,5020000.000,5952500.000,7000000.000,8616911.000,13000000.000
private_equity,302.000,9294675.977,2718676.089,5000822.000,6771556.250,9763022.000,11404500.000,14999999.000
seed,119.000,6964385.613,1864351.984,5017863.000,5750000.000,6250000.000,7602340.000,13000000.000
venture,11353.000,8893183.449,2535430.050,5000001.000,6700000.000,8500000.000,10540450.000,14999999.000


In [52]:
master_frame.groupby(by='funding_round_type')['raised_amount_usd'].agg(['sum'])

,sum
funding_round_type,
angel,269440089.000
private_equity,2806992145.000
seed,828761888.000
venture,100964311691.000


In [53]:
master_frame.funding_round_type.isnull().value_counts()

False    11810
Name: funding_round_type, dtype: int64

In [54]:
master_frame.raised_amount_usd.isnull().value_counts()

False    11810
Name: raised_amount_usd, dtype: int64

In [55]:
master_frame.groupby(by='funding_round_type')['raised_amount_usd'].sum().sort_values(ascending=False)

funding_round_type
venture          100964311691.000
private_equity     2806992145.000
seed                828761888.000
angel               269440089.000
Name: raised_amount_usd, dtype: float64

In [56]:
master_frame.funding_round_type.value_counts()

venture           11353
private_equity      302
seed                119
angel                36
Name: funding_round_type, dtype: int64

In [57]:
master_mean_df = master_frame.pivot_table('raised_amount_usd', 'funding_round_type', aggfunc='mean')['raised_amount_usd'].sort_values(ascending=False)
master_median_df = master_frame.pivot_table('raised_amount_usd', 'funding_round_type', aggfunc='median')['raised_amount_usd'].sort_values(ascending=False)
master_sum_df = master_frame.pivot_table('raised_amount_usd', 'funding_round_type', aggfunc='sum')['raised_amount_usd'].sort_values(ascending=False)
master_count_df = master_frame.pivot_table('raised_amount_usd', 'funding_round_type', aggfunc='count')['raised_amount_usd'].sort_values(ascending=False)

In [58]:
# Lets have a look what can we understand from data, 
fig = make_subplots(1,4)
fig.add_trace(go.Bar(name='Number of investments for each investment type', x=master_count_df.index, y=master_count_df, marker_color = px.colors.qualitative.Plotly[0]), row=1, col=1)
fig.add_trace(go.Bar(name='Sum of amount for the investments for each investment type', x=master_sum_df.index, y=master_sum_df, marker_color = px.colors.qualitative.Plotly[1]), row=1, col=2)
fig.add_trace(go.Bar(name='Average Mean amount of the investments for each investment type', x=master_mean_df.index, y=master_mean_df, marker_color = px.colors.qualitative.Plotly[2]), row=1, col=3)
fig.add_trace(go.Bar(name='Average Median amount of the investments for each investment type', x=master_mean_df.index, y=master_mean_df, marker_color = px.colors.qualitative.Plotly[3]), row=1, col=4)
fig.update_layout(title=dict(text='Number of investments, Sum of amount of investments, Average mean and median of the amount invested for each investment type'), barmode='stack')
fig.show()

In [59]:
# From below we can see that most of the investments across different companies has happed for the Venture investment type
fig = px.box(master_frame,x='funding_round_type', y="raised_amount_usd",color="country_code")
fig.show()

In [60]:
fig = go.Figure(data=[
    go.Bar(name='Count of investments for each investment type',x=master_count_df.index, y=master_count_df),
    go.Bar(name='Sum of investments for each investment type',x=master_sum_df.index, y=master_sum_df),
    go.Bar(name='Mean of investments for each investment type',x=master_mean_df.index, y=master_mean_df)
])
fig.show()

In [61]:
fig = px.box(master_frame,x='funding_round_type', y="raised_amount_usd", color='funding_round_type')
fig.show()

In [62]:
master_frame.groupby(by='funding_round_type')['raised_amount_usd'].agg(['describe'])

describe              ...                          
                       count        mean  ...          75%          max
funding_round_type                        ...                          
angel                 36.000 7484446.917  ...  8616911.000 13000000.000
private_equity       302.000 9294675.977  ... 11404500.000 14999999.000
seed                 119.000 6964385.613  ...  7602340.000 13000000.000
venture            11353.000 8893183.449  ... 10540450.000 14999999.000

[4 rows x 8 columns]

In [63]:
master_frame.groupby(by='funding_round_type')['raised_amount_usd'].agg(['sum'])

,sum
funding_round_type,
angel,269440089.000
private_equity,2806992145.000
seed,828761888.000
venture,100964311691.000


### Q1 Representative funding amount of venture type

In [64]:
venture_df = master_frame[master_frame.funding_round_type=='venture']
print('The raised amount for venture funding is ', venture_df.raised_amount_usd.mean())
millify(venture_df.raised_amount_usd.mean())

The raised amount for venture funding is  8893183.44851581


'9 Million'

In [65]:
# look if there are any outliers
fig = px.box(venture_df,x='funding_round_type', y="raised_amount_usd")
fig.show()

In [66]:
venture_df.raised_amount_usd.describe()

count      11353.000
mean     8893183.449
std      2535430.050
min      5000001.000
25%      6700000.000
50%      8500000.000
75%     10540450.000
max     14999999.000
Name: raised_amount_usd, dtype: float64

### Q2 Representative funding amount of angel type

In [67]:
angel_df = master_frame[master_frame.funding_round_type=='angel']
print('The raised amount for angel funding is ', angel_df.raised_amount_usd.mean())
millify(angel_df.raised_amount_usd.mean())

The raised amount for angel funding is  7484446.916666667


'7 Million'

In [68]:
# look if there are any outliers
fig = px.box(angel_df,x='funding_round_type', y="raised_amount_usd")
fig.show()

In [69]:
angel_df.raised_amount_usd.describe()

count         36.000
mean     7484446.917
std      2120123.503
min      5020000.000
25%      5952500.000
50%      7000000.000
75%      8616911.000
max     13000000.000
Name: raised_amount_usd, dtype: float64

In [70]:
millify(angel_df.raised_amount_usd.mean())

'7 Million'

### Q3 Representative funding amount of seed type

In [71]:
seed_df = master_frame[master_frame.funding_round_type=='seed']
print('The raised amount for seed funding is ', seed_df.raised_amount_usd.mean())
millify(seed_df.raised_amount_usd.mean())

The raised amount for seed funding is  6964385.613445378


'7 Million'

In [72]:
# look if there are any outliers
fig = px.box(seed_df,x='funding_round_type', y="raised_amount_usd")
fig.show()

In [73]:
Q1, Q3, IQR = get_iqr_range(seed_df, 'raised_amount_usd')
print(Q1, Q3, IQR)
seed_df2 = seed_df[~((seed_df.raised_amount_usd < (Q1 - 1.5 * IQR)) |(seed_df.raised_amount_usd > (Q3 + 1.5 * IQR)))]

5750000.0 7602340.0 1852340.0


In [74]:
# look if there are any outliers
fig = px.box(seed_df2,x='funding_round_type', y="raised_amount_usd")
fig.show()

In [75]:
seed_df2.raised_amount_usd.describe()

count        111.000
mean     6600557.550
std      1308975.295
min      5017863.000
25%      5700000.000
50%      6100000.000
75%      7050000.000
max     10000000.000
Name: raised_amount_usd, dtype: float64

In [76]:
millify(seed_df2.raised_amount_usd.mean())

'7 Million'

### Q4 Representative funding amount of private equity type

In [77]:
private_equity_df = master_frame[master_frame.funding_round_type=='private_equity']
print('The raised amount for private equity funding is ', private_equity_df.raised_amount_usd.mean())
millify(private_equity_df.raised_amount_usd.mean())

The raised amount for private equity funding is  9294675.976821192


'9 Million'

In [78]:
private_equity_df.shape

(302, 9)

In [79]:
private_equity_df.describe()

,raised_amount_usd
count,302.000
mean,9294675.977
std,2718676.089
min,5000822.000
25%,6771556.250
50%,9763022.000
75%,11404500.000
max,14999999.000


In [80]:
# look if there are any outliers
fig = px.box(private_equity_df,x='funding_round_type', y="raised_amount_usd")
fig.show()

In [81]:
millify(private_equity_df.raised_amount_usd.mean())

'9 Million'

### Country analysis

#### Creating top 9 for venture investment type

In [82]:
# Creating a grouped df having countries against the total amount of investments for venture funding type.
top9 = venture_df.groupby(['country_code'])['raised_amount_usd'].sum().sort_values(ascending=False).head(9)
top9 = pd.DataFrame(top9).reset_index()

In [83]:
top9

,country_code,raised_amount_usd
0,USA,89006584223.000
1,GBR,4889078691.000
2,CAN,3159289960.000
3,IND,2174543602.000
4,SGP,521143978.000
5,IRL,475855587.000
6,AUS,441989729.000
7,NZL,107672621.000
8,NGA,71000000.000


In [84]:
# A plot showing the top 9 countries against the total amount of investments of venture funding type.
fig = px.bar(top9,x='country_code', y="raised_amount_usd", color='country_code', labels={'country_code':'Country Code', 'raised_amount_usd':'Total Raised Amount'})
fig.update_layout(autosize=False,width=750,height=450,legend=dict(
        x=0.73,
        y=0.99,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))
fig.show() #to check if there are any outliers and i can see that for US  there are some

In [85]:
#Top English-speaking country
print('Top English-speaking country: ',top9.country_code.loc[0])

#Second English-speaking country
print('Second English-speaking country: ',top9.country_code.loc[1])

#Third English-speaking country as CAN
print('Third English-speaking country: ',top9.country_code.loc[2])

Top English-speaking country:  USA
Second English-speaking country:  GBR
Third English-speaking country:  CAN


In [86]:
top9_countries_df = venture_df[venture_df.country_code.isin(top9.country_code)]
top9_countries_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code
0,/ORGANIZATION/-FAME,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.000,/ORGANIZATION/-FAME,#fame,Media,IND
12,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,Analytics,USA
103,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,Service Providers|Technology,USA
112,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
113,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA


In [87]:
pd.DataFrame(top9)

,country_code,raised_amount_usd
0,USA,89006584223.000
1,GBR,4889078691.000
2,CAN,3159289960.000
3,IND,2174543602.000
4,SGP,521143978.000
5,IRL,475855587.000
6,AUS,441989729.000
7,NZL,107672621.000
8,NGA,71000000.000


In [88]:
type_country_invest_df = top9_countries_df[top9_countries_df.country_code.isin(['USA','GBR','CAN'])]

In [89]:
type_country_invest_df.reset_index(drop=True, inplace=True)

In [90]:
type_country_invest_df.head(10)

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,Analytics,USA
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,Service Providers|Technology,USA
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
5,/ORGANIZATION/170-SYSTEMS,/funding-round/b84bb882ca873f5fb96535671981196d,venture,16-04-2002,14000000.000,/ORGANIZATION/170-SYSTEMS,170 Systems,Software,USA
6,/ORGANIZATION/17ZUOYE,/funding-round/8d87f771e938e0f31641bd600abbafca,venture,01-09-2013,10000000.000,/ORGANIZATION/17ZUOYE,17zuoye,Education|Language Learning,USA
7,/ORGANIZATION/2080-MEDIA,/funding-round/281939941d4818948129b8349dbf5f14,venture,30-11-2015,8000000.000,/ORGANIZATION/2080-MEDIA,2080 Media,News,USA
8,/ORGANIZATION/2080-MEDIA,/funding-round/fbbda7a2eacc73185353890b9028797c,venture,06-09-2011,7200000.000,/ORGANIZATION/2080-MEDIA,2080 Media,News,USA
9,/ORGANIZATION/21E6,/funding-round/bdf9f5bf67ee51155eae223acac57ec5,venture,17-11-2013,5050000.000,/ORGANIZATION/21E6,21 Inc,Big Data|Bitcoin|Hardware + Software|Technology,USA


### Sectors

In [91]:
sectors.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1


In [92]:
sectors.shape

(688, 10)

In [93]:
sectors.category_list.nunique()

687

In [94]:
sectors.category_list.isnull().sum()

1

In [95]:
sectors[sectors.category_list.isnull()]

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
0,NaN,0,1,0,0,0,0,0,0,0


In [96]:
sectors.dropna(subset=['category_list'], inplace=True)

In [97]:
sectors.category_list.isnull().sum()

0

In [98]:
sectors.category_list = sectors.category_list.replace({'0':'na', '2.na' :'2.0'}, regex=True)
sectors.head()

,category_list,Automotive & Sports,Blanks,Cleantech / Semiconductors,Entertainment,Health,Manufacturing,"News, Search and Messaging",Others,"Social, Finance, Analytics, Advertising"
1,3D,0,0,0,0,0,1,0,0,0
2,3D Printing,0,0,0,0,0,1,0,0,0
3,3D Technology,0,0,0,0,0,1,0,0,0
4,Accounting,0,0,0,0,0,0,0,0,1
5,Active Lifestyle,0,0,0,0,1,0,0,0,0


In [99]:
#Reshaping the mapping dataframe to merge with the master_frame dataframe. Using melt() function to unpivot the table.
sectors = pd.melt(sectors, id_vars =['category_list'], value_vars =['Manufacturing','Automotive & Sports',
                                                              'Cleantech / Semiconductors','Entertainment',
                                                             'Health','News, Search and Messaging','Others',
                                                             'Social, Finance, Analytics, Advertising']) 
sectors = sectors[~(sectors.value == 0)]
sectors = sectors.drop('value', axis = 1)
sectors = sectors.rename(columns = {"variable":"main_sector"})
sectors.head()


,category_list,main_sector
0,3D,Manufacturing
1,3D Printing,Manufacturing
2,3D Technology,Manufacturing
6,Advanced Materials,Manufacturing
14,Agriculture,Manufacturing


In [100]:
type_country_invest_df.shape

(10911, 9)

In [101]:
type_country_invest_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,Analytics,USA
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,Service Providers|Technology,USA
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA


In [102]:
#check if any value has | in it
type_country_invest_df[type_country_invest_df['category_list'].str.find('|')>-1]

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,Service Providers|Technology,USA
6,/ORGANIZATION/17ZUOYE,/funding-round/8d87f771e938e0f31641bd600abbafca,venture,01-09-2013,10000000.000,/ORGANIZATION/17ZUOYE,17zuoye,Education|Language Learning,USA
9,/ORGANIZATION/21E6,/funding-round/bdf9f5bf67ee51155eae223acac57ec5,venture,17-11-2013,5050000.000,/ORGANIZATION/21E6,21 Inc,Big Data|Bitcoin|Hardware + Software|Technology,USA
11,/ORGANIZATION/23ANDME,/funding-round/97062e0d50c2955cbe40bf8220d87b08,venture,07-01-2011,9000000.000,/ORGANIZATION/23ANDME,23andMe,Biotechnology|Search,USA
12,/ORGANIZATION/23ANDME,/funding-round/c27c1b420dfb34677a6ba449524f1c88,venture,18-06-2009,12600000.000,/ORGANIZATION/23ANDME,23andMe,Biotechnology|Search,USA
...,...,...,...,...,...,...,...,...,...
10898,/ORGANIZATION/ZYGA-TECHNOLOGY,/funding-round/00d2d8d0c604892594b4bd25cf7e18b8,venture,16-12-2013,10000000.000,/ORGANIZATION/ZYGA-TECHNOLOGY,Zyga Technology,Health Care|Medical Devices,USA
10899,/ORGANIZATION/ZYGA-TECHNOLOGY,/funding-round/b7561b5b751428e473d118cae37f776d,venture,06-10-2008,5750000.000,/ORGANIZATION/ZYGA-TECHNOLOGY,Zyga Technology,Health Care|Medical Devices,USA
10904,/ORGANIZATION/ZYNGA,/funding-round/5bebd7acdfd44138f81d162f21c532f9,venture,12-10-2010,6337786.000,/ORGANIZATION/ZYNGA,Zynga,Facebook Applications|Games|Networking|Technology,USA
10905,/ORGANIZATION/ZYNGA,/funding-round/66d64809546fe2031d3479161e3c88f9,venture,01-01-2008,10000000.000,/ORGANIZATION/ZYNGA,Zynga,Facebook Applications|Games|Networking|Technology,USA


In [103]:
# Remove | from the values
#type_country_invest_df["category_list"] = type_country_invest_df["category_list"].str.split("|", n = 1, expand = True)[0]

type_country_invest_df.category_list = type_country_invest_df.category_list.apply(lambda x: x.split('|')[0])

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [104]:
# Check if there are still any values present where | are present
type_country_invest_df[type_country_invest_df['category_list'].str.find('|')>-1]

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code


In [105]:
type_country_invest_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,Analytics,USA
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,Service Providers,USA
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,Manufacturing,USA


In [106]:
type_country_invest_df.category_list.nunique()

413

In [107]:
# covert the category_list strings to lower case
sectors.category_list = sectors.category_list.apply(lambda x: x.lower())
type_country_invest_df["category_list"] = type_country_invest_df["category_list"].str.lower()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [108]:
venture_countries_sectors_df  = pd.merge(type_country_invest_df,sectors, on='category_list', how='left')

In [109]:
venture_countries_sectors_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code,main_sector
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,analytics,USA,"Social, Finance, Analytics, Advertising"
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,service providers,USA,Others
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing


In [110]:
venture_countries_sectors_df.isnull().sum()

company_permalink          0
funding_round_permalink    0
funding_round_type         0
funded_at                  0
raised_amount_usd          0
permalink                  0
name                       0
category_list              0
country_code               0
main_sector                1
dtype: int64

In [111]:
venture_countries_sectors_df.dropna(subset=['main_sector'], inplace=True)

In [112]:
venture_countries_sectors_df.isnull().sum()

company_permalink          0
funding_round_permalink    0
funding_round_type         0
funded_at                  0
raised_amount_usd          0
permalink                  0
name                       0
category_list              0
country_code               0
main_sector                0
dtype: int64

In [113]:
venture_countries_sectors_df.head()

,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code,main_sector
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,analytics,USA,"Social, Finance, Analytics, Advertising"
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,service providers,USA,Others
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing


In [114]:
# FT: Venture type, Country: USA
D1 = venture_countries_sectors_df[(venture_countries_sectors_df['country_code'] == 'USA')]
D1_gr = D1[['raised_amount_usd','main_sector']].groupby('main_sector').agg(['sum', 'count']).rename(
    columns={'sum':'Total_amount','count' : 'Total_count'})
D1 = D1.merge(D1_gr, how='left', on ='main_sector')
D1.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code,main_sector,"(raised_amount_usd, Total_amount)","(raised_amount_usd, Total_count)"
0,/ORGANIZATION/0XDATA,/funding-round/e1cfcbe1bdf4c70277c5f29a3482f24e,venture,19-07-2014,8900000.000,/ORGANIZATION/0XDATA,H2O.ai,analytics,USA,"Social, Finance, Analytics, Advertising",19397376964.000,2216
1,/ORGANIZATION/128-TECHNOLOGY,/funding-round/fb6216a30cb566ede89e0bee0623a634,venture,16-12-2014,11999347.000,/ORGANIZATION/128-TECHNOLOGY,128 Technology,service providers,USA,Others,21736007002.000,2423
2,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/786f61aa9866f4471151285f5c56be36,venture,03-02-2010,5150000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing,6228553378.000,691
3,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/82ace97530965cd2be8f262836b43ff5,venture,27-03-2008,12400000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing,6228553378.000,691
4,/ORGANIZATION/1366-TECHNOLOGIES,/funding-round/ab99fc5a53717b1b53fd6aa5687c5fa9,venture,16-12-2010,6000000.000,/ORGANIZATION/1366-TECHNOLOGIES,1366 Technologies,manufacturing,USA,Manufacturing,6228553378.000,691


In [115]:
# FT: Venture type, Country: GBR
D2 = venture_countries_sectors_df[(venture_countries_sectors_df['country_code'] == 'GBR')]
D2_gr = D2[['raised_amount_usd','main_sector']].groupby('main_sector').agg(['sum', 'count']).rename(
    columns={'sum':'Total_amount','count' : 'Total_count'})
D2 = D2.merge(D2_gr, how='left', on ='main_sector')
D2.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code,main_sector,"(raised_amount_usd, Total_amount)","(raised_amount_usd, Total_count)"
0,/ORGANIZATION/365SCORES,/funding-round/48212f931f542fdef78810bc87aef086,venture,29-09-2014,5500000.000,/ORGANIZATION/365SCORES,365Scores,android,GBR,"Social, Finance, Analytics, Advertising",974404014.000,118
1,/ORGANIZATION/7DIGITAL,/funding-round/b5ad7ed7baddd3974bd51403f17dd88f,venture,01-01-2008,8468328.000,/ORGANIZATION/7DIGITAL,7digital,content creators,GBR,Entertainment,422784687.000,48
2,/ORGANIZATION/7DIGITAL,/funding-round/eafacfcceb1fbc4fd605f641b603313e,venture,19-10-2012,10000000.000,/ORGANIZATION/7DIGITAL,7digital,content creators,GBR,Entertainment,422784687.000,48
3,/ORGANIZATION/90MIN,/funding-round/bd626ed022f5c66574b1afe234f3c90d,venture,07-05-2013,5800000.000,/ORGANIZATION/90MIN,90min,media,GBR,Entertainment,422784687.000,48
4,/ORGANIZATION/ABCODIA,/funding-round/3d20c23d203134ed86c0d1b2bec288b2,venture,18-05-2015,8259067.000,/ORGANIZATION/ABCODIA,Abcodia,biotechnology,GBR,Cleantech / Semiconductors,1123990056.000,126


In [116]:
# FT: Venture type, Country: CAN
D3 = venture_countries_sectors_df[(venture_countries_sectors_df['country_code'] == 'CAN')]
D3_gr = D3[['raised_amount_usd','main_sector']].groupby('main_sector').agg(['sum', 'count']).rename(
    columns={'sum':'Total_amount','count' : 'Total_count'})
D3 = D3.merge(D3_gr, how='left', on ='main_sector')
D3.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/merge.py:643: UserWarning:

merging between different levels can give an unintended result (1 levels on the left,2 on the right)



,company_permalink,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd,permalink,name,category_list,country_code,main_sector,"(raised_amount_usd, Total_amount)","(raised_amount_usd, Total_count)"
0,/ORGANIZATION/2CATALYZE,/funding-round/d26e31e33cf217e2ed888eafc54febaf,venture,25-08-2010,6785876.000,/ORGANIZATION/2CATALYZE,2Catalyze,software,CAN,Others,851137962.000,98
1,/ORGANIZATION/360INCENTIVES-COM,/funding-round/8712b14c58f7066491d136673931ed3e,venture,15-01-2013,7650000.000,/ORGANIZATION/360INCENTIVES-COM,360incentives.com,software,CAN,Others,851137962.000,98
2,/ORGANIZATION/500PX,/funding-round/344b1e2b60ff76f4548c69ea7c04901f,venture,07-08-2013,8800000.000,/ORGANIZATION/500PX,500px,apps,CAN,"News, Search and Messaging",350121113.000,40
3,/ORGANIZATION/500PX,/funding-round/36b2c6975d20b602b5d9ea9724e383ea,venture,21-07-2015,13000000.000,/ORGANIZATION/500PX,500px,apps,CAN,"News, Search and Messaging",350121113.000,40
4,/ORGANIZATION/6N-SILICON,/funding-round/82a8781150acd315eee75194fab51c21,venture,14-07-2007,5714286.000,/ORGANIZATION/6N-SILICON,6N Silicon,concentrated solar power,CAN,Cleantech / Semiconductors,905887607.000,98


In [117]:
#Total number of investments (count)
print(D1.raised_amount_usd.count())
print(D2.raised_amount_usd.count())
print(D3.raised_amount_usd.count())

9983
561
366


In [118]:
#Total amount of investment (USD)
print(round(D1.raised_amount_usd.sum(), 2))
print(round(D2.raised_amount_usd.sum(), 2))
print(round(D3.raised_amount_usd.sum(), 2))

88997097294.0
4889078691.0
3159289960.0


In [119]:
#Top sector, second-top, third-top for D1 (based on count of investments)
#Number of investments in the top, second-top, third-top sector in D1
D1_num_investments =pd.DataFrame(D1_gr['raised_amount_usd']['Total_count'].sort_values(ascending=False))
D1_num_investments['country_code'] = 'USA'
D1_num_investments[:3]

,Total_count,country_code
main_sector,,
Others,2423,USA
"Social, Finance, Analytics, Advertising",2216,USA
Cleantech / Semiconductors,1991,USA


In [120]:
#Top sector, second-top, third-top for D2 (based on count of investments)
#Number of investments in the top, second-top, third-top sector in D2
D2_num_investments = pd.DataFrame(D2_gr['raised_amount_usd']['Total_count'].sort_values(ascending=False))
D2_num_investments['country_code'] = 'GBR'
D2_num_investments[:3]

,Total_count,country_code
main_sector,,
Others,129,GBR
Cleantech / Semiconductors,126,GBR
"Social, Finance, Analytics, Advertising",118,GBR


In [121]:
#Top sector, second-top, third-top for D3 (based on count of investments)
#Number of investments in the top, second-top, third-top sector in D3
D3_num_investments = pd.DataFrame(D3_gr['raised_amount_usd']['Total_count'].sort_values(ascending=False))
D3_num_investments['country_code'] = 'CAN'
D3_num_investments[:3]

,Total_count,country_code
main_sector,,
Others,98,CAN
Cleantech / Semiconductors,98,CAN
"Social, Finance, Analytics, Advertising",69,CAN


In [122]:
#For the top sector USA , which company received the highest investment?
company = D1[D1['main_sector']=='Others']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
print(company)
company = company.sort_values(by = ['raised_amount_usd'], ascending = False).head()
print(company.head(1))

#For the second top sector USA , which company received the highest investment?
company = D1[D1['main_sector']=='Social, Finance, Analytics, Advertising']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
company = company.sort_values(by = 'raised_amount_usd', ascending = False).head()
print(company.head(1))

#Check if the names are same for all the permalinks
print('\n')
print(D1[D1.company_permalink=='/ORGANIZATION/AIRTIGHT-NETWORKS']['name'].unique())
print(D1[D1.company_permalink=='/ORGANIZATION/SHOTSPOTTER']['name'].unique())

                                   raised_amount_usd
company_permalink                                   
/ORGANIZATION/128-TECHNOLOGY            11999347.000
/ORGANIZATION/170-SYSTEMS               14000000.000
/ORGANIZATION/17ZUOYE                   10000000.000
/ORGANIZATION/2U                        15100000.000
/ORGANIZATION/3CROWD-TECHNOLOGIES        6620000.000
...                                              ...
/ORGANIZATION/ZMANDA                     8000000.000
/ORGANIZATION/ZOLA                      10000000.000
/ORGANIZATION/ZOZI                      10000000.000
/ORGANIZATION/ZUDY                       7500000.000
/ORGANIZATION/ZUNIVERSITY-COM           10000000.000

[1796 rows x 1 columns]
                                 raised_amount_usd
company_permalink                                 
/ORGANIZATION/AIRTIGHT-NETWORKS       54201907.000
                           raised_amount_usd
company_permalink                           
/ORGANIZATION/SHOTSPOTTER       67933006.00

In [123]:
#For the top sector GBR , which company received the highest investment?
company = D2[D2['main_sector']=='Others']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
company = company.sort_values(by = 'raised_amount_usd', ascending = False).head()
print(company.head(1))

#For the second top sector GBR , which company received the highest investment?
company = D2[D2['main_sector']=='Cleantech / Semiconductors']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
company = company.sort_values(by = 'raised_amount_usd', ascending = False).head()
print(company.head(1))


#Check if the names are same for all the permalinks
print('\n')
print(D2[D2.company_permalink=='/ORGANIZATION/ELECTRIC-CLOUD']['name'].unique())
print(D2[D2.company_permalink=='/ORGANIZATION/EUSA-PHARMA']['name'].unique())

                              raised_amount_usd
company_permalink                              
/ORGANIZATION/ELECTRIC-CLOUD       37000000.000
                           raised_amount_usd
company_permalink                           
/ORGANIZATION/EUSA-PHARMA       35600000.000


['Electric Cloud']
['EUSA Pharma']


In [124]:
#For the top sector CAN , which company received the highest investment?
company = D3[D3['main_sector']=='Cleantech / Semiconductors']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
company = company.sort_values(by = 'raised_amount_usd', ascending = False).head()
print(company.head(1))

#For the second top sector CAN , which company received the highest investment?
company = D3[D3['main_sector']=='Others']
company = company.pivot_table(values = 'raised_amount_usd', index = 'company_permalink', aggfunc = 'sum')
company = company.sort_values(by = 'raised_amount_usd', ascending = False).head()
print(company.head(1))

#Check if the names are same for all the permalinks
print('\n')
print(D3[D3.company_permalink=='/ORGANIZATION/FRESCO-MICROCHIP']['name'].unique())
print(D3[D3.company_permalink=='/ORGANIZATION/NEWSTEP']['name'].unique())

                                raised_amount_usd
company_permalink                                
/ORGANIZATION/FRESCO-MICROCHIP       48000000.000
                       raised_amount_usd
company_permalink                       
/ORGANIZATION/NEWSTEP       31477853.000


['Fresco Microchip']
['NewStep Networks']


In [125]:
# Concatenate the df having number of investments in each sector for selected countries
top3 = pd.concat([D1_num_investments[:3],D2_num_investments[:3],D3_num_investments[:3]])
top3.reset_index(inplace=True)

### Further visualizations

In [126]:
# A plot showing the number of investments in the top 3 sectors of the top 3 countries
fig = px.bar(y='Total_count', x='country_code', color='main_sector', data_frame=top3, barmode='group',labels={'Total_count':'Number of Investments','country_code':'Country Code',"main_sector": 'Sector'})
fig.update_layout(autosize=False,width=750,height=450,legend=dict(
        x=0.5,
        y=0.99,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))
fig.show()

In [127]:
fig = px.pie(top3, values='Total_count', names='country_code', title='% Number of investments in top 3 countries')
fig.update_layout(autosize=False,width=450,height=450,legend=dict(
        x=0.95,
        y=0.99,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))
fig.show()

In [128]:
fig = px.area(top3, x="country_code", y="Total_count", color="main_sector",
	      line_group="country_code", title='% Number of investments in top 3 countries across top 3 sectors')
fig.update_layout(autosize=False,width=700,height=450,legend=dict(
        x=.4,
        y=0.99,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
    ))
fig.show()

In [129]:
top3

,main_sector,Total_count,country_code
0,Others,2423,USA
1,"Social, Finance, Analytics, Advertising",2216,USA
2,Cleantech / Semiconductors,1991,USA
3,Others,129,GBR
4,Cleantech / Semiconductors,126,GBR
5,"Social, Finance, Analytics, Advertising",118,GBR
6,Others,98,CAN
7,Cleantech / Semiconductors,98,CAN
8,"Social, Finance, Analytics, Advertising",69,CAN
